In [9]:
import numpy as np

def pairwise_squared_euclidean_distance(X, Y):
    # ||x_i - y_j||^2 = ||x_i||^2 + ||y_j||^2 - 2 * <x_i,y_j>
    # Computing the squared norms of matrices X and Y
    X_norm_squared = np.sum(X**2, axis=1).reshape(-1, 1)  # getting norm of X as a column vector of shape =(m, 1)
    Y_norm_squared = np.sum(Y**2, axis=1).reshape(1, -1)  # getting norm of Y as a row vector of shape = (1, n)

    # Computing the dot product of X and Y
    X_Y_dot_product = np.dot(X, Y.T)

    return X_norm_squared + Y_norm_squared - 2 * X_Y_dot_product


X = np.random.randn(5, 3)  # 5 x 3 matrix
Y = np.random.randn(6, 3)  # 6 x 3 matrix

Z = pairwise_squared_euclidean_distance(X, Y)

print(Z)


[[ 3.78310818  4.27767705  8.71328193  5.51970771  0.79353336  5.75576211]
 [ 7.4115149   8.32538897  4.81733086  2.29226973  0.40959987  2.2072255 ]
 [ 8.87944675 12.00922436  8.51876617 15.10596834  6.6372873  11.73220874]
 [16.36751849 19.75629397  4.84628674  2.30843453  3.11250521  1.70013291]
 [13.6578931  17.89966908 14.58391046  5.68664168  3.46964349  6.94653735]]


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [11]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

# Step 1: Load the Caltech-101 Dataset
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform)  # Replace with your data path

# Step 2: Split data into training and testing sets by indices
dataset_size = len(dataset)
indices = list(range(dataset_size))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Step 3: Create DataLoaders for train and test sets
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Step 4: Load Pre-trained ResNet-50 Model
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove the last classification layer
resnet50.eval()  # Set model to evaluation mode

# Step 5: Extract Feature Embeddings (same as before)
def extract_features(data_loader, model, device):
    model = model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten the output tensor
            features.append(outputs.cpu().numpy())
            labels.append(targets.cpu().numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_features, train_labels = extract_features(train_loader, resnet50, device)
test_features, test_labels = extract_features(test_loader, resnet50, device)

# Step 6: k-NN Classification
knn = KNeighborsClassifier(n_neighbors=5)  # You can tune k


KeyboardInterrupt: 

In [6]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Load the Caltech-101 Dataset and Split into Train/Test Sets
# Assuming you have downloaded the dataset and it is organized in 'caltech101' directory.
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the entire dataset
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform)

# Split dataset into training (2/3) and testing (1/3)
train_size = int(2/3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Step 2: Load Pre-trained ResNet-50 Model
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove the last classification layer
resnet50.eval()  # Set model to evaluation mode

# Step 3: Extract Feature Embeddings
def extract_features(data_loader, model, device):
    model = model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten the output tensor
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_features, train_labels = extract_features(train_loader, resnet50, device)
test_features, test_labels = extract_features(test_loader, resnet50, device)

# Step 4: k-NN Classification
knn = KNeighborsClassifier(n_neighbors=5)  # You can tune k
knn.fit(train_features, train_labels)

# Step 5: Evaluate on Test Set
test_predictions = knn.predict(test_features)
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Test Set Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 194MB/s]


KeyboardInterrupt: 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Load and Split the Caltech-101 Dataset
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the entire dataset
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform)

# Split dataset into training (2/3) and testing (1/3)
train_size = int(2 / 3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Step 2: Load Pre-trained ResNet-50 Model
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove the last classification layer
resnet50.eval()  # Set model to evaluation mode

# Step 3: Extract Feature Embeddings
def extract_features(data_loader, model, device):
    model = model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten the output tensor
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_features, train_labels = extract_features(train_loader, resnet50, device)
test_features, test_labels = extract_features(test_loader, resnet50, device)

# Step 4: Train Linear Classifier
# Convert features and labels into torch tensors
train_features_tensor = torch.tensor(train_features, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)

# Define the linear classifier (multi-class logistic regression)
num_classes = len(dataset.classes)
input_dim = train_features.shape[1]

classifier = nn.Linear(input_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Training loop for linear classifier
epochs = 20
batch_size = 32
num_samples = train_features_tensor.shape[0]

for epoch in range(epochs):
    classifier.train()
    total_loss = 0

    # Mini-batch training
    permutation = torch.randperm(num_samples)
    for i in range(0, num_samples, batch_size):
        indices = permutation[i:i + batch_size]
        batch_features = train_features_tensor[indices]
        batch_labels = train_labels_tensor[indices]

        # Forward pass
        outputs = classifier(batch_features)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Step 5: Evaluate on Test Set
classifier.eval()
test_features_tensor = torch.tensor(test_features, dtype=torch.float32).to(device)

with torch.no_grad():
    test_outputs = classifier(test_features_tensor)
    _, test_predictions = torch.max(test_outputs, 1)

test_accuracy = accuracy_score(test_labels, test_predictions.cpu().numpy())
print(f"Test Set Accuracy: {test_accuracy * 100:.2f}%")


KeyboardInterrupt: 

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Step 1: Load and Split the Caltech-101 Dataset with Data Augmentation
# Data augmentation for better generalization
data_transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the entire dataset and split it into training and testing
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform_train)
train_size = int(2 / 3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Apply test transform to the test dataset
test_dataset.dataset.transform = data_transform_test

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Step 2: Load Pre-trained ResNet-50 Model and Modify for Fine-tuning
resnet50 = models.resnet50(pretrained=True)

# Modify the fully connected layer to match the number of classes in Caltech-101
num_classes = len(dataset.classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)

# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.0001)  # Use a small learning rate for fine-tuning
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Learning rate scheduler

# Step 4: Train the Network
epochs = 10
for epoch in range(epochs):
    resnet50.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()  # Adjust the learning rate
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Step 5: Evaluate on Test Set
resnet50.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet50(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Set Accuracy: {test_accuracy:.2f}%")


KeyboardInterrupt: 